# (3) Using Pydantic with SQL

Pydantic can integrate with a lot of existing tools:

- SQL (<https://sqlmodel.tiangolo.com>)
- FastAPI (<https://fastapi.tiangolo.com>)
- Pandera (<https://pandera.readthedocs.io/en/stable/pydantic_integration.html>)

In this notebook we will explore SQLModel.

First we need to define our models. They are similar to models we defined in notebook 2, but have some additional syntax.

In [1]:
from typing import List, Optional

from rich.pretty import pprint
from sqlmodel import Field, Relationship, Session, SQLModel, create_engine, select

# %% Define the models
class Book(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    title: str
    publisher: str
    year: int
    author_id: Optional[int] = Field(default=None, foreign_key="author.id")
    author: Optional["Author"] = Relationship(back_populates="books")


class Author(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    year_of_birth: int
    books: Optional[List[Book]] = Relationship(back_populates="author")


engine = create_engine("sqlite:///database.db")
SQLModel.metadata.create_all(engine)

Now, lets create some data. SQLModel + Pydantic give us a really nice and readable way to create and interact with data.

In [2]:
authors = [
    sam_edwardes := Author(name="Sam Edwardes", year_of_birth=1993),
    olivia_edwardes := Author(name="Olivia Edwardes", year_of_birth=1998)
]

books = [
    book_1 := Book(title="book 1", publisher="O Willy", year=2023, author=sam_edwardes),
    book_2 := Book(title="book 2", publisher="O Willy", year=2023, author=sam_edwardes),
]

In [3]:
sam_edwardes

Author(id=None, name='Sam Edwardes', year_of_birth=1993, books=[Book(id=None, title='book 1', publisher='O Willy', year=2023, author_id=None, author=Author(id=None, name='Sam Edwardes', year_of_birth=1993, books=[...])), Book(id=None, title='book 2', publisher='O Willy', year=2023, author_id=None, author=Author(id=None, name='Sam Edwardes', year_of_birth=1993, books=[...]))])

In [4]:
book_2

Book(id=None, title='book 2', publisher='O Willy', year=2023, author_id=None, author=Author(id=None, name='Sam Edwardes', year_of_birth=1993, books=[Book(id=None, title='book 1', publisher='O Willy', year=2023, author_id=None, author=Author(id=None, name='Sam Edwardes', year_of_birth=1993, books=[...])), Book(id=None, title='book 2', publisher='O Willy', year=2023, author_id=None, author=Author(id=None, name='Sam Edwardes', year_of_birth=1993, books=[...]))]))

Next insert the data into the database.

In [5]:
with Session(engine) as session:
    for book in books:
        # Note that we only need to add the book. Since the book is related to
        # an author the author will also be inserted!
        session.add(book)
    session.commit()

Now lets get the data back from the database.

In [6]:
with Session(engine) as session:
    statement = select(Book)
    sql_data = session.exec(statement).all()
    pprint(sql_data)

[
│   Book(year=2023, publisher='O Willy', id=1, title='book 1', author_id=1),
│   Book(year=2023, publisher='O Willy', id=2, title='book 2', author_id=1)
]

That was nice, I got back a list of books. All of the data has been validated by Pydantic, and I can access the data via attributes.

In [7]:
for book in sql_data:
    print(f"{book.title} - {book.year}")

book 1 - 2023
book 2 - 2023


One downside is I only have the author id. I do not have any details about them.

In [8]:
sql_data[0].author_id

1

In [9]:
try:
    sql_data[0].author
except Exception as e:
    print(e)

Parent instance <Book at 0x121e46350> is not bound to a Session; lazy load operation of attribute 'author' cannot proceed (Background on this error at: https://sqlalche.me/e/14/bhk3)


But... if I keep my connection to the database open I can access this data!

In [10]:
with Session(engine) as session:
    statement = select(Book)
    books = session.exec(statement).all()
    
    for book in books:
        print("=" * 32)
        print(book.title)
        print(book.author)

book 1
year_of_birth=1993 id=1 name='Sam Edwardes'
book 2
year_of_birth=1993 id=1 name='Sam Edwardes'


I can also use the relationship in reverse. I can query authors to get all of the books they have written.

In [11]:
with Session(engine) as session:
    statement = select(Author).where(Author.name == "Sam Edwardes")
    author = session.exec(statement).one_or_none()
    pprint(author)
    pprint(author.books)

Author(year_of_birth=1993, id=1, name='Sam Edwardes')

[
│   Book(year=2023, publisher='O Willy', id=1, title='book 1', author_id=1),
│   Book(year=2023, publisher='O Willy', id=2, title='book 2', author_id=1)
]